In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import pickle

In [ ]:
feature_columns = ['f_'+str(i) for i in range(0,300)]

**Load Model from ubiquant_cur_feature_without_inv_id_no_preprocess**

In [ ]:
model_path = '../input/ubiquant-cur-feature-without-inv-id-no-preprocess/'
weights_path = '../input/ubiquant-cur-feature-without-inv-id-no-preprocess/'
model_name_pearson_pickle_path = '../input/ubiquant-cur-feature-without-inv-id-no-preprocess/model_name_pearson.pickle'
with open(model_name_pearson_pickle_path,'rb') as input_file:
    model_name_pearson = pickle.load(input_file)
num_of_models = len(model_name_pearson)
def predict(test_df_with_features):
    
    predictions = []
    for index,row in model_name_pearson.iterrows():
        # Load model architecture
        model_arch = row[0]
        model_file = model_path+model_arch
        weights_file = model_file+'.h5'
        json_file = open(model_file, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = tf.keras.models.model_from_json(loaded_model_json)
        # Load model weights 
        model.load_weights(weights_file)
        predictions.append(model.predict(test_df_with_features).reshape(-1))
    predictions = np.concatenate(predictions,axis=0).reshape(num_of_models,-1)
    return np.median(predictions,axis=0).reshape(-1)

**Submission**

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    test_df_with_features = test_df.fillna(0)
    test_df_with_features = test_df_with_features[feature_columns].values
    predictions = predict(test_df_with_features)
    predictions = np.reshape(predictions,(-1))
    sample_prediction_df['target'] = predictions  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions